In [225]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse

In [226]:
def distance(x1, y1, x2, y2):
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)

In [227]:
df_path = "./tobii_test_sub_updv2_mixed_face_pred.csv"
df_path_golden = "./golden_test.csv"
user_specs_path = "../data/participant_characteristics.csv"
user_specs = pd.read_csv(user_specs_path)[[
    "Display Height (pixels)", "Display Width (pixels)", 'Participant ID',
    'Screen Width (cm)', 'Screen Height (cm)', 'Distance From Screen (cm)',
    'Self-Reported Vision'
]]
df_pred = pd.read_csv(df_path)
df_pred["distances"] = df_pred[['x_normalized', 'y_normalized', 'pred_x', 'pred_y']].apply(lambda x: distance(*x), axis = 1)
df_pred["user_id"] = df_pred["full_name"].apply(lambda x: x.split("#")[0])
df_pred = df_pred.merge(user_specs, left_on = "user_id", right_on = 'Participant ID', how = "inner").drop(columns = ['Participant ID'])

df_golden = pd.read_csv(df_path_golden)
df_pred_golden = df_golden.merge(df_pred, on = "paths", how = "inner", suffixes=["", "_"])

In [228]:
df_pred.head()

,paths,x_normalized,y_normalized,full_name,eyes_exist,pred_x,pred_y,distances,user_id,Display Height (pixels),Display Width (pixels),Screen Width (cm),Screen Height (cm),Distance From Screen (cm),Self-Reported Vision
0,/home/ubuntu/projects/tweakle/data/tobii_image...,0.727410,0.433413,P_25#1492110782158_14_-study-educational_advan...,True,0.738195,0.409692,0.026058,P_25,1200,1920,51.7,32.31,78.0,Glasses
1,/home/ubuntu/projects/tweakle/data/tobii_image...,0.336066,0.420706,P_25#1492110782158_10_-study-benefits_of_runni...,True,0.434957,0.410425,0.099424,P_25,1200,1920,51.7,32.31,78.0,Glasses
2,/home/ubuntu/projects/tweakle/data/tobii_image...,0.442917,0.399263,P_25#1492110782158_10_-study-benefits_of_runni...,True,0.422824,0.402419,0.020339,P_25,1200,1920,51.7,32.31,78.0,Glasses
3,/home/ubuntu/projects/tweakle/data/tobii_image...,0.489065,0.366477,P_25#1492110782158_14_-study-educational_advan...,True,0.434435,0.423249,0.078788,P_25,1200,1920,51.7,32.31,78.0,Glasses
4,/home/ubuntu/projects/tweakle/data/tobii_image...,0.225222,0.423625,P_25#1492110782158_14_-study-educational_advan...,True,0.335141,0.412617,0.110469,P_25,1200,1920,51.7,32.31,78.0,Glasses


In [229]:
df_pred["Self-Reported Vision"].value_counts()

Self-Reported Vision
Normal      7792
Glasses     4508
Contacts    1489
Name: count, dtype: int64

In [230]:
def average_angle(df):
    df_ = df.copy()
    df_["x"] = df_["x_normalized"] * df_["Screen Width (cm)"]
    df_["pred_x_"] = df_["pred_x"] * df_["Screen Width (cm)"]
    df_["y"] = df_["y_normalized"] * df_["Screen Height (cm)"]
    df_["pred_y_"] = df_["pred_y"] * df_["Screen Height (cm)"]
    df_["center_y"] = df_["Screen Height (cm)"] * 0.5
    df_["center_x"] = df_["Screen Width (cm)"] * 0.5
    df_["distances_center_pred"] = df_[['center_x', 'center_y', 'pred_x_', 'pred_y_']].apply(lambda x: distance(*x), axis = 1)
    df_["distances_center_gt"] = df_[['center_x', 'center_y', 'x', 'y']].apply(lambda x: distance(*x), axis = 1)
    df_["tg_center_gt"] = df_["distances_center_gt"] / df_["Distance From Screen (cm)"]
    df_["tg_center_pred"] = df_["distances_center_pred"] / df_["Distance From Screen (cm)"]
    df_["angle_gt"] = np.arctan(df_["tg_center_gt"].values) * 180 / np.pi
    df_["angle_pred"] = np.arctan(df_["tg_center_pred"].values) * 180 / np.pi
    angles = (df_["angle_gt"] - df_["angle_pred"]).abs()
    return angles.mean(), angles.std()

In [231]:
def average_pixel_distance(df):
    df_ = df.copy()
    df_["x"] = df_["x_normalized"] * df_["Display Width (pixels)"]
    df_["pred_x_"] = df_["pred_x"] * df_["Display Width (pixels)"]
    df_["y"] = df_["y_normalized"] * df_["Display Height (pixels)"]
    df_["pred_y_"] = df_["pred_y"] * df_["Display Height (pixels)"]
    df_["distances"] = df_[['x', 'y', 'pred_x_', 'pred_y_']].apply(lambda x: distance(*x), axis = 1)
    return df_["distances"].mean(), df_["distances"].std()

In [232]:
def compute_metrics(df):
    preds = df[["pred_x", "pred_y"]].values
    labels = df[["x_normalized", "y_normalized"]].values
    
    mape_value = mape(labels, preds)
    print(f"Test MAPE: {round(mape_value, 4)}")
    
    rmse_value = mse(labels, preds, squared=True)
    print(f"Test RMSE: {round(rmse_value, 4)}")
    
    av_rel_dist = df["distances"].mean()
    print(f"Average relative distance: {round(av_rel_dist, 4)} +- {round(df['distances'].std(), 3)}")

    av_pixel_dist, std = average_pixel_distance(df)
    print(f"Average pixel distance: {round(av_pixel_dist, 1)} +- {round(std, 1)}")
    av_angle, std = average_angle(df)
    print(f"Average angle: {round(av_angle, 2)} +- {round(std, 2)}")

All test

In [233]:
compute_metrics(df_pred)

Test MAPE: 0.4353
Test RMSE: 0.0165
Average relative distance: 0.1451 +- 0.109
Average pixel distance: 208.7 +- 157.4
Average angle: 2.83 +- 2.53


Glasses

In [234]:
df_glasses = df_pred[df_pred['Self-Reported Vision'] == "Glasses"]

In [235]:
compute_metrics(df_glasses)

Test MAPE: 0.475
Test RMSE: 0.0177
Average relative distance: 0.1504 +- 0.113
Average pixel distance: 217.3 +- 163.9
Average angle: 2.93 +- 2.75


Normal

In [236]:
df_glasses = df_pred[df_pred['Self-Reported Vision'] != "Glasses"]

In [237]:
compute_metrics(df_glasses)

Test MAPE: 0.416
Test RMSE: 0.0159
Average relative distance: 0.1425 +- 0.107
Average pixel distance: 204.5 +- 154.0
Average angle: 2.78 +- 2.42


Golden test

In [238]:
compute_metrics(df_pred_golden)

Test MAPE: 0.4166
Test RMSE: 0.0174
Average relative distance: 0.1609 +- 0.095
Average pixel distance: 225.7 +- 136.3
Average angle: 3.11 +- 2.43
